In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
engine = create_engine('sqlite:///../banking.db')

In [3]:
# SQL queries
queries = {
    "Total Transaction Volume & Fraud Count": """
    SELECT
      COUNT(*) AS total_transactions,
      SUM(CASE WHEN isFraud = 1 THEN 1 ELSE 0 END) AS total_fraud_transactions
    FROM transactions;
    """,

    "Transactions by Type": """
    SELECT
      type AS transaction_type,
      COUNT(*) AS transaction_count,
      SUM(amount) AS total_amount
    FROM transactions
    GROUP BY transaction_type
    ORDER BY total_amount DESC;
    """,

    "Fraud Rate by Transaction Type": """
    SELECT
      type AS transaction_type,
      ROUND(SUM(CASE WHEN isFraud = 1 THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS fraud_rate_percentage
    FROM transactions
    GROUP BY transaction_type
    ORDER BY fraud_rate_percentage DESC;
    """,

    "Hourly Transaction Trends": """
    SELECT
      step,
      COUNT(*) AS total_transactions,
      SUM(amount) AS total_amount
    FROM transactions
    GROUP BY step
    ORDER BY step;
    """,

    "Top Accounts by Transaction Volume": """
    SELECT
      nameOrig AS account,
      SUM(amount) AS total_sent
    FROM transactions
    GROUP BY account
    ORDER BY total_sent DESC
    LIMIT 10;
    """
}


In [4]:
results = {}

for query_name, query in queries.items():
    df = pd.read_sql_query(query, engine)
    results[query_name] = df
    print(f"--- {query_name} ---")
    display(df)

--- Total Transaction Volume & Fraud Count ---


,total_transactions,total_fraud_transactions
0,6362620,8213


--- Transactions by Type ---


,transaction_type,transaction_count,total_amount
0,TRANSFER,532909,4.852920e+11
1,CASH_OUT,2237500,3.944130e+11
2,CASH_IN,1399284,2.363674e+11
3,PAYMENT,2151495,2.809337e+10
4,DEBIT,41432,2.271992e+08


--- Fraud Rate by Transaction Type ---


,transaction_type,fraud_rate_percentage
0,TRANSFER,0.77
1,CASH_OUT,0.18
2,PAYMENT,0.00
3,DEBIT,0.00
4,CASH_IN,0.00


--- Hourly Transaction Trends ---


,step,total_transactions,total_amount
0,1,2708,2.854292e+08
1,2,1014,8.592160e+07
2,3,552,4.329388e+07
3,4,565,7.291003e+07
4,5,665,4.554809e+07
...,...,...,...
738,739,10,1.658783e+07
739,740,6,7.632964e+06
740,741,22,8.782899e+07
741,742,14,1.432374e+07


--- Top Accounts by Transaction Volume ---


,account,total_sent
0,C1715283297,92445516.64
1,C2127282686,73823490.36
2,C2044643633,71172480.42
3,C1425667947,69886731.30
4,C1584456031,69337316.27
5,C811810230,67500761.29
6,C420748282,66761272.21
7,C1139847449,64234448.19
8,C300140823,63847992.58
9,C372535854,63294839.63


In [5]:
for query_name, df in results.items():
    filename = f"../data/processed/{query_name.replace(' ', '_').lower()}.csv"
    df.to_csv(filename, index=False)
    print(f"Saved {filename}")

Saved ../data/processed/total_transaction_volume_&_fraud_count.csv
Saved ../data/processed/transactions_by_type.csv
Saved ../data/processed/fraud_rate_by_transaction_type.csv
Saved ../data/processed/hourly_transaction_trends.csv
Saved ../data/processed/top_accounts_by_transaction_volume.csv
